In [ ]:
from __future__ import division
import glob
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
from scipy.linalg import eig
from sklearn.linear_model import Ridge
%matplotlib inline

In [ ]:
#Load speech commands data from dir

rootDir = 'Lyon_Speech_Commands\*'
paths = glob.glob(rootDir)

data = []

for n,i in enumerate(paths):
    print 'loading ' + paths[n][21:]
    block = []
    files = glob.glob(i + '\*')
    
    for j in files:
        X = np.loadtxt(j)
        block.append(X)
        
    data.append(block)

In [ ]:
# generate the ESN reservoir weights
a = 0.3 # leaking rate

# set input weights and scaling parameter
W_in = 1 * (np.random.rand(2000, 31)-0.5)

# set reservoir weights
W = (np.random.rand(2000, 2000)-0.5)

# set spectral radius parameter
rhoW = max(abs(eig(W)[0]))
W *= 1.25 / rhoW

In [ ]:
#Simulate the reservoir

#C are integrated reservoir states, T are targets
C = []
T = []

#randomly pick category and data point
for itr in xrange(5000):
    R = []
    r = np.zeros(2000)
    ctg = np.random.randint(1, 30)
    rnd = np.random.randint(0,1500)
    
    #run the reservoir with the current data point
    for t in xrange(np.shape(data[ctg][rnd])[1]):
        x = data[ctg][rnd][:,t]
        r = (1-a)*r + a*np.tanh(np.dot(W_in, x) + np.dot(W, r))
        R.append(r)
        
    #integrate the reservoir state over time
    R_int = np.sum(R, axis=0)
    
    #add reservoir state and target to C
    C.append(R_int)
    T.append(ctg)
    
    if itr%100 == 0:
        print 'iteration number: ' + str(itr)

In [ ]:
#format targets to 1-hot encoding
Targets = np.zeros((len(T), 31))

for i, vec in enumerate(Targets):
    Targets[i][T[i]] = 1
    
print Targets[0]
print Targets[-1]
print Targets[200]

In [ ]:
#Train ridge regression model

clf = Ridge(alpha = 0.5)
clf.fit(C, Targets)

In [ ]:
#run reservoir on validation data

C = []
T = []
r = np.zeros(2000)

for itr in xrange(500):
    R = []
    ctg = np.random.randint(1, 30)
    rnd = np.random.randint(1500,1700)
    
    for t in xrange(np.shape(data[ctg][rnd])[1]):
        x = data[ctg][rnd][:,t]
        r = (1-a)*r + a*np.tanh(np.dot(W_in, x) + np.dot(W, r))
        R.append(r)
        
    R = np.array(R)        
    R_int = np.sum(R, axis=0)
    C.append(R_int)
    T.append(ctg)

In [ ]:
Targets = np.zeros((len(T), 31))

for i, vec in enumerate(Targets):
    Targets[i][T[i]] = 1

In [ ]:
#determine accuracy
score = 0
for i,n in enumerate(C):
    y_pred = clf.predict(C[i].reshape(1,-1))
    y = np.argmax(y_pred)
    if y == np.argmax(Targets[i]):
        score += 1

print len(C)
print score/len(C)